# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 10 2022 8:30AM,248213,10,0086064952 (C7062262),ISDIN Corporation,Cancelled
1,Oct 10 2022 8:30AM,248213,10,0086064962,ISDIN Corporation,Released
2,Oct 10 2022 8:30AM,248213,10,0086064963,ISDIN Corporation,Released
3,Oct 10 2022 8:30AM,248213,10,0086064960,ISDIN Corporation,Released
4,Oct 10 2022 8:30AM,248213,10,0086064974,ISDIN Corporation,Released
5,Oct 10 2022 8:30AM,248213,10,0086064973,ISDIN Corporation,Released
6,Oct 10 2022 8:30AM,248213,10,0086064977,ISDIN Corporation,Released
7,Oct 10 2022 8:30AM,248213,10,0086064979,ISDIN Corporation,Released
8,Oct 10 2022 8:30AM,248213,10,0086064980,ISDIN Corporation,Released
9,Oct 10 2022 8:30AM,248213,10,0086064982,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
13,248210,Released,1
14,248213,Cancelled,1
15,248213,Released,22
16,248214,Released,1
17,248215,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Completed,Released
id,,,
248202,NaN,NaN,1.0
248210,NaN,NaN,1.0
248213,1.0,NaN,22.0
248214,NaN,NaN,1.0
248215,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Released
id,,,
248168,0.0,0.0,1.0
248172,1.0,0.0,0.0
248184,0.0,0.0,1.0
248187,0.0,0.0,1.0
248191,0.0,11.0,4.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Executing does not exist yet


ShipmentStatus,Cancelled,Completed,Released
id,,,
248168,0.0,0,1
248172,1.0,0,0
248184,0.0,0,1
248187,0.0,0,1
248191,0.0,11,4


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Released
0,248168,0.0,0,1
1,248172,1.0,0,0
2,248184,0.0,0,1
3,248187,0.0,0,1
4,248191,0.0,11,4


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Executing does not exist yet


ShipmentStatus,id,Cancelled,Completed,Released
0,248168,0.0,,1
1,248172,1.0,,
2,248184,0.0,,1
3,248187,0.0,,1
4,248191,0.0,11,4


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Completed     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 10 2022 8:30AM,248213,10,ISDIN Corporation
23,Oct 10 2022 8:30AM,248214,10,ISDIN Corporation
24,Oct 10 2022 8:30AM,248215,10,"Nextsource Biotechnology, LLC"
25,Oct 7 2022 6:01PM,248196,21,"NBTY Global, Inc."
26,Oct 7 2022 5:59PM,248194,21,"NBTY Global, Inc."
27,Oct 7 2022 2:15PM,248210,10,"ClearSpec, LLC"
28,Oct 7 2022 1:40PM,248202,20,"Adare Pharmaceuticals, Inc."
29,Oct 7 2022 1:28PM,248193,15,"Person & Covey, Inc."
44,Oct 7 2022 1:25PM,248192,15,"Person & Covey, Inc."
59,Oct 7 2022 1:24PM,248191,15,"Person & Covey, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Completed,Released
0,Oct 10 2022 8:30AM,248213,10,ISDIN Corporation,1.0,,22
1,Oct 10 2022 8:30AM,248214,10,ISDIN Corporation,0.0,,1
2,Oct 10 2022 8:30AM,248215,10,"Nextsource Biotechnology, LLC",0.0,,1
3,Oct 7 2022 6:01PM,248196,21,"NBTY Global, Inc.",0.0,,1
4,Oct 7 2022 5:59PM,248194,21,"NBTY Global, Inc.",0.0,,1
5,Oct 7 2022 2:15PM,248210,10,"ClearSpec, LLC",0.0,,1
6,Oct 7 2022 1:40PM,248202,20,"Adare Pharmaceuticals, Inc.",0.0,,1
7,Oct 7 2022 1:28PM,248193,15,"Person & Covey, Inc.",0.0,11,4
8,Oct 7 2022 1:25PM,248192,15,"Person & Covey, Inc.",0.0,14,1
9,Oct 7 2022 1:24PM,248191,15,"Person & Covey, Inc.",0.0,11,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Completed
0,Oct 10 2022 8:30AM,248213,10,ISDIN Corporation,22,
1,Oct 10 2022 8:30AM,248214,10,ISDIN Corporation,1,
2,Oct 10 2022 8:30AM,248215,10,"Nextsource Biotechnology, LLC",1,
3,Oct 7 2022 6:01PM,248196,21,"NBTY Global, Inc.",1,
4,Oct 7 2022 5:59PM,248194,21,"NBTY Global, Inc.",1,
5,Oct 7 2022 2:15PM,248210,10,"ClearSpec, LLC",1,
6,Oct 7 2022 1:40PM,248202,20,"Adare Pharmaceuticals, Inc.",1,
7,Oct 7 2022 1:28PM,248193,15,"Person & Covey, Inc.",4,11
8,Oct 7 2022 1:25PM,248192,15,"Person & Covey, Inc.",1,14
9,Oct 7 2022 1:24PM,248191,15,"Person & Covey, Inc.",4,11


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Completed
0,Oct 10 2022 8:30AM,248213,10,ISDIN Corporation,22,
1,Oct 10 2022 8:30AM,248214,10,ISDIN Corporation,1,
2,Oct 10 2022 8:30AM,248215,10,"Nextsource Biotechnology, LLC",1,
3,Oct 7 2022 6:01PM,248196,21,"NBTY Global, Inc.",1,
4,Oct 7 2022 5:59PM,248194,21,"NBTY Global, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Executing does not exist yet


C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,Date,id,Warehouse,Customer,Released,Completed
0,Oct 10 2022 8:30AM,248213,10,ISDIN Corporation,22.0,NaN
1,Oct 10 2022 8:30AM,248214,10,ISDIN Corporation,1.0,NaN
2,Oct 10 2022 8:30AM,248215,10,"Nextsource Biotechnology, LLC",1.0,NaN
3,Oct 7 2022 6:01PM,248196,21,"NBTY Global, Inc.",1.0,NaN
4,Oct 7 2022 5:59PM,248194,21,"NBTY Global, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Completed
0,Oct 10 2022 8:30AM,248213,10,ISDIN Corporation,22.0,0.0
1,Oct 10 2022 8:30AM,248214,10,ISDIN Corporation,1.0,0.0
2,Oct 10 2022 8:30AM,248215,10,"Nextsource Biotechnology, LLC",1.0,0.0
3,Oct 7 2022 6:01PM,248196,21,"NBTY Global, Inc.",1.0,0.0
4,Oct 7 2022 5:59PM,248194,21,"NBTY Global, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Completed
Warehouse,,,
10,1241036,26.0,0.0
15,744576,9.0,36.0
16,248187,1.0,0.0
18,248172,0.0,0.0
20,496370,2.0,0.0
21,496390,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Completed
0,10,1241036,26.0,0.0
1,15,744576,9.0,36.0
2,16,248187,1.0,0.0
3,18,248172,0.0,0.0
4,20,496370,2.0,0.0
5,21,496390,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Completed
0,10,26.0,0.0
1,15,9.0,36.0
2,16,1.0,0.0
3,18,0.0,0.0
4,20,2.0,0.0
5,21,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,26.0
1,15,Released,9.0
2,16,Released,1.0
3,18,Released,0.0
4,20,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,20,21
Status,,,,,,
Completed,0.0,36.0,0.0,0.0,0.0,0.0
Released,26.0,9.0,1.0,0.0,2.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,20,21
0,Completed,0.0,36.0,0.0,0.0,0.0,0.0
1,Released,26.0,9.0,1.0,0.0,2.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,20,21
0,Completed,0.0,36.0,0.0,0.0,0.0,0.0
1,Released,26.0,9.0,1.0,0.0,2.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()